# Importing Libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.models import load_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
print(tf.__version__)

# Data preprocessing

In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory= r"../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                       target_size=(224,224))

tsdata = ImageDataGenerator()
validdata = tsdata.flow_from_directory(directory=r"../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                     target_size=(224,224))

In [ ]:
label_map = (traindata.class_indices) #classes and their indices
print(label_map)

# Creating VGG16 model

In [ ]:
vgg_16 = VGG16(include_top=False, weights='imagenet',input_shape=(224, 224, 3))

# Freeze all the layers, as non-trainable
for layer in vgg_16.layers[:]:
    layer.trainable = False

#Model summary
vgg_16.summary()

Adding Layers to the model

In [ ]:
model = Sequential()

# Add the vgg base 
model.add(vgg_16)
print("Shape:",model.output_shape)

# Add new layers
model.add(Flatten())
print("Shape:",model.output_shape)

model.add(Dense(38, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()


Compiling the model

In [ ]:
#Compile
model.compile(loss='categorical_crossentropy',
      optimizer= optimizers.Adam(lr=0.00001),
      metrics=["acc"])

Adding Callbacks

In [ ]:
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=50)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.000001, verbose=1)
cp = ModelCheckpoint(filepath="/kaggle/working/best_weights.hdf5", 
                               monitor = 'val_acc',
                               verbose=1, 
                               save_best_only=True)

# Training the model

In [ ]:
history = model.fit(
    traindata,
    batch_size=500,
    epochs=1,
    callbacks=[es,rlr, cp],
    validation_data=validdata)

# Save model and weights

In [ ]:
model.load_weights(r'../input/savedmodel/best_weights.hdf5')
model.save(r'/kaggle/working/best-model.h5')

# Loss summary

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()


plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# Test Data

In [ ]:
model = load_model('../input/savedmodel/best-model.h5')

In [ ]:
def testimage(img):    
    plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    class_index = model.predict_classes(images, batch_size=10)
    for name, index in label_map.items():
        if index == class_index[0]:
            print(name)

Load test image and call the function 

In [ ]:
img = image.load_img(r'../input/new-plant-diseases-dataset/test/test/TomatoHealthy3.JPG', target_size=(224, 224))
testimage(img)

Histogram Equalization

In [ ]:
sourceimg = np.copy(img)
R, G, B = cv2.split(sourceimg)

output1_R = cv2.equalizeHist(R)
output1_G = cv2.equalizeHist(G)
output1_B = cv2.equalizeHist(B)

equ = cv2.merge((output1_R, output1_G, output1_B))
testimage(equ)

# Evaluation

Confusion Matrix

In [ ]:
Y_pred = model.predict_generator(validdata)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = (confusion_matrix(validdata.classes, y_pred))

for i in cm:
    print (i)

# Colour Segmentation

For arbitrary datasets

In [ ]:
hsv = cv2.cvtColor(sourceimg, cv2.COLOR_RGB2BGR)
hsv = cv2.cvtColor(sourceimg, cv2.COLOR_BGR2HSV)

# find the green color 
mask_green = cv2.inRange(hsv, (36,0,0), (86,255,255))
# find the brown color
mask_brown = cv2.inRange(hsv, (8, 60, 20), (30, 255, 200))
# find the yellow color in the leaf
mask_yellow = cv2.inRange(hsv, (21, 39, 64), (40, 255, 255))

# find any of the three colors(green or brown or yellow) in the image
mask = cv2.bitwise_or(mask_green, mask_brown)
mask = cv2.bitwise_or(mask, mask_yellow)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(sourceimg,sourceimg, mask=mask)
plt.imshow(res)